In [ ]:
# Function (tools) calling

In [1]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
!export PYTHONPATH=":./python"

In [2]:
from devtools import debug
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool

from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

### Step 1 : Ask LLM to return an instantiated function  

In [5]:
#llm = get_llm("mistral_large_edenai")
llm = get_llm("gpt_4omini_edenai")
#llm = get_llm("gpt_35_azure")
messages = []


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    print("'add' tool called")
    return a + b


llm_with_tools = llm.bind_tools([add])
prompt = def_prompt(
    "Use the provided functions  to compute what is 11 + 12; execute the function "
)
chain = prompt | llm_with_tools
ai_msg = chain.invoke({})

debug(ai_msg)

2024-09-05 10:15:08.888 | INFO     | python.ai_core.llm:get_llm:319 - get LLM:'gpt_4omini_edenai' -configurable: False - streaming: False
2024-09-05 10:15:08.896 | INFO     | python.ai_core.cache:set_cache:54 - LLM cache : SQLiteCache


/tmp/ipykernel_505959/1624562191.py:26 <module>
    ai_msg: AIMessage(
        content='',
        response_metadata={
            'mistral': {
                'generated_text': '',
                'message': [
                    {
                        'role': 'user',
                        'message': '',
                        'tools': [
                            {
                                'name': 'add',
                                'description': (
                                    'Adds a and b.\n'
                                    '\n'
                                    '    Args:\n'
                                    '        a: first int\n'
                                    '        b: second int'
                                ),
                                'parameters': {
                                    'type': 'object',
                                    'properties': {
                                        'a': {
                         

AIMessage(content='', response_metadata={'mistral': {'generated_text': '', 'message': [{'role': 'user', 'message': '', 'tools': [{'name': 'add', 'description': 'Adds a and b.\n\n    Args:\n        a: first int\n        b: second int', 'parameters': {'type': 'object', 'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b']}}], 'tool_calls': None}, {'role': 'assistant', 'message': '', 'tools': None, 'tool_calls': [{'id': 'EY1Tcek8T', 'name': 'add', 'arguments': '{"a": 11, "b": 12}'}]}], 'status': 'success', 'cost': 0.003384}}, id='run-753322a8-87b1-4358-95f9-f8f5113af453-0', tool_calls=[{'name': 'add', 'args': {'a': 11, 'b': 12}, 'id': 'EY1Tcek8T', 'type': 'tool_call'}])

In [ ]:
debug(llm_with_tools)

In [ ]:
# Step 2 : Execute the function, and send result to LLM

In [6]:
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

r = llm_with_tools.invoke(messages)

debug(r)

'add' tool called
/tmp/ipykernel_505959/3057862763.py:11 <module>
    r: AIMessage(
        content='The sum of 11 and 12 is 23.',
        response_metadata={
            'openai': {
                'generated_text': 'The sum of 11 and 12 is 23.',
                'message': [
                    {
                        'role': 'user',
                        'message': '',
                        'tools': [
                            {
                                'name': 'add',
                                'description': (
                                    'Adds a and b.\n'
                                    '\n'
                                    '    Args:\n'
                                    '        a: first int\n'
                                    '        b: second int'
                                ),
                                'parameters': {
                                    'type': 'object',
                                    'properties': {
      

AIMessage(content='The sum of 11 and 12 is 23.', response_metadata={'openai': {'generated_text': 'The sum of 11 and 12 is 23.', 'message': [{'role': 'user', 'message': '', 'tools': [{'name': 'add', 'description': 'Adds a and b.\n\n    Args:\n        a: first int\n        b: second int', 'parameters': {'type': 'object', 'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b']}}], 'tool_calls': None}, {'role': 'assistant', 'message': 'The sum of 11 and 12 is 23.', 'tools': None, 'tool_calls': []}], 'status': 'success', 'cost': 2.52e-05}}, id='run-95dfd48f-9a3c-4554-b140-965c8d16ec57-0')

###  Other method : Use a tool calling agent

In [8]:
import math

from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate

llm = get_llm("gpt_35_azure")

@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def exponentiate(x: float, y: float) -> float:
    """Calculate the power of a number. Return x**y (w to the power of y)"""
    print("exponentiate")
    return math.pow(x, y)


# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Math Assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [add, exponentiate]
agent = create_tool_calling_agent(llm, tools, prompt)  # type: ignore
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)  # type: ignore

r = agent_executor.invoke({"input": "what is 12  + 100^3"})
debug(r)

2024-09-05 10:17:59.172 | INFO     | python.ai_core.llm:get_llm:319 - get LLM:'gpt_35_azure' -configurable: False - streaming: False


2024-09-05 10:17:59.290 | INFO     | python.ai_core.cache:set_cache:54 - LLM cache : SQLiteCache




> Entering new AgentExecutor chain...
Could not parse tool input: {'arguments': '// Using the add and exponentiate functions from the previous code block\n\nconst result = functions.add({\n  a: 12,\n  b: functions.exponentiate({\n    x: 100,\n    y: 3,\n  }),\n});\n\nresult', 'name': 'python'} because the `arguments` is not valid JSON.Invalid or incomplete responseTo solve `12 + 100^3`, we can use the `add` and `exponentiate` functions from the previous code block.

```
const result = functions.add({
  a: 12,
  b: functions.exponentiate({
    x: 100,
    y: 3,
  }),
});

console.log(result); // Output: 1000012
```

Therefore, `12 + 100^3 = 1000012`.

> Finished chain.
/tmp/ipykernel_505959/3866373677.py:40 <module>
    r: {
        'input': 'what is 12  + 100^3',
        'output': (
            'To solve `12 + 100^3`, we can use the `add` and `exponentiate` functions from the previous code block.\n'
            '\n'
            '```\n'
            'const result = functions.add({\n'
 

{'input': 'what is 12  + 100^3',
 'output': 'To solve `12 + 100^3`, we can use the `add` and `exponentiate` functions from the previous code block.\n\n```\nconst result = functions.add({\n  a: 12,\n  b: functions.exponentiate({\n    x: 100,\n    y: 3,\n  }),\n});\n\nconsole.log(result); // Output: 1000012\n```\n\nTherefore, `12 + 100^3 = 1000012`.'}

In [ ]:
## Call Eden.ai provided tools